In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


This shows us where the competition data is stored, so that we can load the files into the notebook.  We'll do that next.

## Load the data

The second code cell in your notebook now appears below the three lines of output with the file locations.

![](https://i.imgur.com/OQBax9n.png)

Type the two lines of code below into your second code cell.  Then, once you're done, either click on the blue play button, or hit **[Shift] + [Enter]**.  

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Your code should return the output above, which corresponds to the first five rows of the table in **train.csv**.  It's very important that you see this output **in your notebook** before proceeding with the tutorial!
> _If your code does not produce this output_, double-check that your code is identical to the two lines above.  And, make sure your cursor is in the code cell before hitting **[Shift] + [Enter]**.

The code that you've just written is in the Python programming language. It uses a Python "module" called **pandas** (abbreviated as `pd`) to load the table from the **train.csv** file into the notebook. To do this, we needed to plug in the location of the file (which we saw was `/kaggle/input/titanic/train.csv`).  
> If you're not already familiar with Python (and pandas), the code shouldn't make sense to you -- but don't worry!  The point of this tutorial is to (quickly!) make your first submission to the competition.  At the end of the tutorial, we suggest resources to continue your learning.

At this point, you should have at least three code cells in your notebook.  
![](https://i.imgur.com/ReLhYca.png)

Copy the code below into the third code cell of your notebook to load the contents of the **test.csv** file.  Don't forget to click on the play button (or hit **[Shift] + [Enter]**)!

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


As before, make sure that you see the output above in your notebook before continuing.  

Once all of the code runs successfully, all of the data (in **train.csv** and **test.csv**) is loaded in the notebook.  (_The code above shows only the first 5 rows of each table, but all of the data is there -- all 891 rows of **train.csv** and all 418 rows of **test.csv**!_)

# Part 3: Your first submission

Remember our goal: we want to find patterns in **train.csv** that help us predict whether the passengers in **test.csv** survived.

It might initially feel overwhelming to look for patterns, when there's so much data to sort through.  So, we'll start simple.

## Explore a pattern

Remember that the sample submission file in **gender_submission.csv** assumes that all female passengers survived (and all male passengers died).  

Is this a reasonable first guess?  We'll check if this pattern holds true in the data (in **train.csv**).

Copy the code below into a new code cell.  Then, run the cell.

In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


Before moving on, make sure that your code returns the output above.  The code above calculates the percentage of female passengers (in **train.csv**) who survived.

Then, run the code below in another code cell:

In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


The code above calculates the percentage of male passengers (in **train.csv**) who survived.

From this you can see that almost 75% of the women on board survived, whereas only 19% of the men lived to tell about it. Since gender seems to be such a strong indicator of survival, the submission file in **gender_submission.csv** is not a bad first guess!

But at the end of the day, this gender-based submission bases its predictions on only a single column.  As you can imagine, by considering multiple columns, we can discover more complex patterns that can potentially yield better-informed predictions.  Since it is quite difficult to consider several columns at once (or, it would take a long time to consider all possible patterns in many different columns simultaneously), we'll use machine learning to automate this for us.

## Your first machine learning model

We'll build what's known as a **random forest model**.  This model is constructed of several "trees" (there are three trees in the picture below, but we'll construct 100!) that will individually consider each passenger's data and vote on whether the individual survived.  Then, the random forest model makes a democratic decision: the outcome with the most votes wins!

![](https://i.imgur.com/AC9Bq63.png)

The code cell below looks for patterns in four different columns (**"Pclass"**, **"Sex"**, **"SibSp"**, and **"Parch"**) of the data.  It constructs the trees in the random forest model based on patterns in the **train.csv** file, before generating predictions for the passengers in **test.csv**.  The code also saves these new predictions in a CSV file **submission.csv**.

Copy this code into your notebook, and run it in a new code cell.

In [6]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


Make sure that your notebook outputs the same message above (`Your submission was successfully saved!`) before moving on.
> Again, don't worry if this code doesn't make sense to you!  For now, we'll focus on how to generate and submit predictions.

Once you're ready, click on the **"Save Version"** button in the top right corner of your notebook.  This will generate a pop-up window.  
- Ensure that the **"Save and Run All"** option is selected, and then click on the **"Save"** button.
- This generates a window in the bottom left corner of the notebook.  After it has finished running, click on the number to the right of the **"Save Version"** button.  This pulls up a list of versions on the right of the screen.  Click on the ellipsis **(...)** to the right of the most recent version, and select **Open in Viewer**.  
- Click on the **Data** tab on the top of the screen.  Then, click on the **"Submit"** button to submit your results.

![](https://i.imgur.com/1ocaUl4.png)

Congratulations for making your first submission to a Kaggle competition!  Within ten minutes, you should receive a message providing your spot on the leaderboard.  Great work!

# Part 4: Learn more!

If you're interested in learning more, we strongly suggest our (3-hour) **[Intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning)** course, which will help you fully understand all of the code that we've presented here.  You'll also know enough to generate even better predictions!